In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ai-trading-bot-using-deep-q-learning/AI trader bot .ipynb


# importing the libararies 

In [4]:
import numpy as np 
import pandas as pd 
import random 
import matplotlib.pyplot as plt 
import tensorflow as tf 
import math 
import pandas_datareader as data_reader 

from tqdm import tqdm_notebook , tqdm 
from collections import deque 

# building the AI trader network

In [5]:
class AI_Trader():
  
  def __init__(self, state_size, action_space=3, model_name="AITrader"): #Stay, Buy, Sell
    
    self.state_size = state_size
    self.action_space = action_space
    self.memory = deque(maxlen=2000)
    self.inventory = []
    self.model_name = model_name
    
    self.gamma = 0.95
    self.epsilon = 1.0
    self.epsilon_final = 0.01
    self.epsilon_decay = 0.995
    
    self.model = self.model_builder()
    
  def model_builder(self):
    
    model = tf.keras.models.Sequential()
    
    model.add(tf.keras.layers.Dense(units=32, activation='relu', input_dim=self.state_size))
    
    model.add(tf.keras.layers.Dense(units=64, activation='relu'))
    
    model.add(tf.keras.layers.Dense(units=128, activation='relu'))
    
    model.add(tf.keras.layers.Dense(units=self.action_space, activation='linear'))
    
    model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(lr=0.001))
    
    return model
  
  def trade(self, state):
    
    if random.random() <= self.epsilon:
      return random.randrange(self.action_space)
    
    actions = self.model.predict(state)
    return np.argmax(actions[0])
  
  
  def batch_train(self, batch_size):
    
    batch = []
    for i in range(len(self.memory) - batch_size + 1, len(self.memory)):
      batch.append(self.memory[i])
      
    for state, action, reward, next_state, done in batch:
      reward = reward
      if not done:
        reward = reward + self.gamma * np.amax(self.model.predict(next_state)[0])
        
      target = self.model.predict(state)
      target[0][action] = reward
      
      self.model.fit(state, target, epochs=1, verbose=0)
      
    if self.epsilon > self.epsilon_final:
      self.epsilon *= self.epsilon_decay
            
        

# data set preprocessing 

## sigmoid 

In [6]:
def sigmoid(x):
    return 1/(1 + math.exp(-x))

## price format function 

In [7]:
def stock_price_format(n):
    if n < 0 :
        return '- $ {:2f}'.format(abs(n))
    else :
        return '$ {:2f}'.format(abs(n))

# dataset loader 

In [8]:
def dataset_loader(stock_name):
    
    dataset = data_reader.DataReader(stock_name , data_source = 'yahoo')
    
    start_date = str(dataset.index[0]).split()[0]
    end_date = str(dataset.index[-1]).split()[0]
    
    close = dataset['Close']
    return close  

In [9]:
# this is for example purpose for apple stocks 


dataset_for_example = data_reader.DataReader('AAPL' , data_source = 'yahoo')
dataset_for_example


,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2015-08-24,108.800003,92.000000,94.870003,103.120003,162206300.0,95.294647
2015-08-25,111.110001,103.500000,111.110001,103.739998,103601600.0,95.867607
2015-08-26,109.889999,105.050003,107.089996,109.690002,96774600.0,101.366074
2015-08-27,113.239998,110.019997,112.230003,112.919998,84616100.0,104.350983
2015-08-28,113.309998,111.540001,112.169998,113.290001,53164400.0,104.692871
...,...,...,...,...,...,...
2020-08-17,464.350006,455.850006,464.250000,458.429993,29890400.0,458.429993
2020-08-18,464.000000,456.029999,457.410004,462.250000,26408400.0,462.250000
2020-08-19,468.649994,462.440002,463.929993,462.829987,36384500.0,462.829987


# state creator 

In [10]:
def state_creator(data, timestep, window_size):
  
  starting_id = timestep - window_size + 1
  
  if starting_id >= 0:
    windowed_data = data[starting_id:timestep+1]
  else:
    windowed_data = - starting_id * [data[0]] + list(data[0:timestep+1])
    
  state = []
  for i in range(window_size - 1):
    state.append(sigmoid(windowed_data[i+1] - windowed_data[i]))
    
  return np.array([state])

# loading a dataset 

In [11]:
stock_name = 'AAPL'
data = dataset_loader(stock_name)

data 

Date
2015-08-24    103.120003
2015-08-25    103.739998
2015-08-26    109.690002
2015-08-27    112.919998
2015-08-28    113.290001
                 ...    
2020-08-17    458.429993
2020-08-18    462.250000
2020-08-19    462.829987
2020-08-20    473.100006
2020-08-21    489.179993
Name: Close, Length: 1259, dtype: float64

# Training the AI trader

## setting the hyper parameters 

In [12]:
window_size = 10 
episodes = 1000

batch_size = 32
data_samples = len(data) - 1 


## defining the trader model 

In [13]:
trader = AI_Trader(window_size)

In [14]:
trader.model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 32)                352       
_________________________________________________________________
dense_1 (Dense)              (None, 64)                2112      
_________________________________________________________________
dense_2 (Dense)              (None, 128)               8320      
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 387       
Total params: 11,171
Trainable params: 11,171
Non-trainable params: 0
_________________________________________________________________


# training loop ( # this training process will take time )

In [ ]:
 
 for episode in range(1, episodes + 1):
  
  print("Episode: {}/{}".format(episode, episodes))
  
  state = state_creator(data, 0, window_size + 1)
  
  total_profit = 0
  trader.inventory = []
  
  for t in tqdm(range(data_samples)):
    
    action = trader.trade(state)
    
    next_state = state_creator(data, t+1, window_size + 1)
    reward = 0
    
    if action == 1: #Buying
      trader.inventory.append(data[t])
      print("AI Trader bought: ", stock_price_format(data[t]))
      
    elif action == 2 and len(trader.inventory) > 0: #Selling
      buy_price = trader.inventory.pop(0)
      
      reward = max(data[t] - buy_price, 0)
      total_profit += data[t] - buy_price
      print("AI Trader sold: ", stock_price_format(data[t]), " Profit: " + stock_price_format(data[t] - buy_price) )
      
    if t == data_samples - 1:
      done = True
    else:
      done = False
      
    trader.memory.append((state, action, reward, next_state, done))
    
    state = next_state
    
    if done:
      print("########################")
      print("TOTAL PROFIT: {}".format(total_profit))
      print("########################")
    
    if len(trader.memory) > batch_size:
      trader.batch_train(batch_size)
      
  if episode % 10 == 0:
    trader.model.save("ai_trader_{}.h5".format(episode))

  0%|          | 0/1258 [00:00<?, ?it/s]

Episode: 1/1000
AI Trader bought:  $ 103.120003
AI Trader sold:  $ 109.690002  Profit: $ 6.570000
AI Trader bought:  $ 109.269997
AI Trader bought:  $ 112.309998
AI Trader sold:  $ 110.150002  Profit: $ 0.880005
AI Trader sold:  $ 112.570000  Profit: $ 0.260002
AI Trader bought:  $ 114.209999
AI Trader sold:  $ 116.279999  Profit: $ 2.070000
AI Trader bought:  $ 113.449997
AI Trader bought:  $ 115.209999
AI Trader sold:  $ 114.320000  Profit: $ 0.870003
AI Trader bought:  $ 115.000000
AI Trader bought:  $ 109.059998
AI Trader bought:  $ 110.379997
AI Trader sold:  $ 111.309998  Profit: - $ 3.900002
AI Trader bought:  $ 110.779999
AI Trader sold:  $ 109.500000  Profit: - $ 5.500000


  3%|▎         | 33/1258 [00:04<03:04,  6.64it/s]

AI Trader bought:  $ 112.120003


  3%|▎         | 34/1258 [00:08<21:07,  1.04s/it]

...

 10%|█         | 132/1258 [05:31<1:00:46,  3.24s/it]

AI Trader sold:  $ 101.500000  Profit: $ 5.400002


 11%|█         | 133/1258 [05:35<1:01:24,  3.27s/it]

AI Trader bought:  $ 103.010002


 11%|█         | 134/1258 [05:38<1:01:23,  3.28s/it]